# Exploring $P(U)$ Further  (with Healpix)

In [61]:
# imports
import os
from pkg_resources import resource_filename

import numpy as np
import pandas

import astropy_healpix

from astropy.table import Table
from astropy.io import fits
from astropy.coordinates import SkyCoord

from astropath import priors
from astropath import path

from frb.frb import FRB

# Config

In [15]:
base_config = dict(
    max_radius=10.,
    cut_size=None,
    deblend=False,
    cand_bright=None,
    cand_separation=None,
    skip_bayesian=False,
    posterior_method='fixed',
    npixels=9,
)
#
# FRB 20201123A (MeerTRAP;  Rajwade+2022)
updates = dict(
    name='FRB20201123A',
    hpix_file = os.path.join(resource_filename('frb', 'data'), 
                             'FRBs', 'healpix',
                             'FRB20201123A_hpix_uniform.fits.gz'),
    cand_file=os.path.join(resource_filename('frb', 'data'),
                           'Galaxies', '20201123A',
                            'FRB20201123A_path_candidates.csv'),
    PU = 0.1, # Prior
    posterior_method='local',
    cut_size = 30.,
    filter = 'GMOS-r',
)

config = {**base_config, **updates} 

# FRB

In [65]:
ifrb = FRB.by_name(config['name'])

# PATH object

In [69]:
path_obj = path.PATH()
path_obj.frb = ifrb

# Load Candidates

In [70]:
# Read
path_obj.candidates = pandas.read_csv(config['cand_file'])
# Coords
path_obj.coords = SkyCoord(ra=path_obj.candidates.ra.values,
                           dec=path_obj.candidates.dec.values,
                           unit='deg')
# Add separation?
if 'separation' not in path_obj.candidates.keys():
    seps = path_obj.frb.coord.separation(path_obj.coords)
    path_obj.candidates['separation'] = seps.to('arcsec').value

## Coords

In [94]:
path_obj.init_cand_coords()

# Priors

In [137]:
prior = priors.load_std_priors()['adopted']
prior['theta']['scale'] = 0.5
prior['U'] = 0.1 # 
# 
prior

{'theta': {'method': 'exp', 'max': 6.0, 'scale': 0.5},
 'O': 'inverse',
 'U': 0.1,
 'name': 'Adopted',
 'nhalf': 10.0}

## Calcs

In [138]:
# Set priors
path_obj.init_cand_prior('inverse', P_U=prior['U'])
path_obj.init_theta_prior(prior['theta']['method'], 
                            prior['theta']['max'],
                            prior['theta']['scale'])


In [139]:
P_O = path_obj.calc_priors()                            
P_O

array([0.8372987 , 0.00591807, 0.00263919, 0.0078417 , 0.00160889,
       0.01425449, 0.03043895])

In [140]:
path_obj.candidates

,ra,dec,ang_size,mag,separation,P_O,p_xO,P_Ox
2,263.65963,-50.76407,1.75,16.041294,27.790394,0.837299,0.000463,0.649412
4,263.65821,-50.76454,0.20,20.431294,31.431113,0.005918,0.000460,0.004560
7,263.65972,-50.76561,0.15,21.261294,30.916794,0.002639,0.000460,0.002034
9,263.66119,-50.76641,0.21,20.151294,30.577024,0.007842,0.000460,0.006042
13,263.66322,-50.76732,0.63,21.791294,30.541401,0.001609,0.000460,0.001240
18,263.66128,-50.76787,0.24,19.571294,34.597137,0.014254,0.000460,0.010984
29,263.66560,-50.77039,0.28,18.861294,38.722546,0.030439,0.000460,0.023454


In [141]:
np.sum(path_obj.candidates.P_O)

0.9

# Localization

In [142]:
# Load healpix
hpix = Table.read(config['hpix_file'])
header = fits.open(config['hpix_file'])[1].header
nside = 2**header['MOCORDER']

## Check normalization

In [143]:
np.sum(hpix['PROBDENSITY'])

52786.0

## Normalize

In [144]:
healpix = astropy_healpix.HEALPix(nside=nside)

In [145]:
healpix.pixel_area.to('arcsec**2').value

0.040520897663894

In [146]:
heal_area = healpix.pixel_area.to('arcsec**2').value

In [147]:
norm =  np.sum(hpix['PROBDENSITY']) *  healpix.pixel_area.to('arcsec**2').value
norm

2138.9361040863087

In [148]:
hpix_norm = hpix.copy()
hpix_norm['PROBDENSITY'] /= norm

## Init

In [149]:
localiz = dict(type='healpix',
            healpix_data=hpix_norm, 
            healpix_nside=nside,
            healpix_ordering='NUNIQ',
            healpix_coord='C')            
path_obj.init_localization('healpix', **localiz)

# Posteriors

## Set max_radius 

#### This should be the box_width of the equivalent area of the uniform localization region

In [150]:
path_obj.max_radius = np.sqrt(heal_area * np.sum(hpix['PROBDENSITY'] > 0)) / 2.
path_obj.max_radius

23.124316768751832

In [151]:
path_obj.candidates.keys()

Index(['ra', 'dec', 'ang_size', 'mag', 'separation', 'P_O', 'p_xO', 'P_Ox'], dtype='object')

In [152]:
P_Ox, P_Ux = path_obj.calc_posteriors('local', 
                         box_hwidth=path_obj.max_radius,
                         max_radius=path_obj.max_radius)
#
P_Ux

0.10097678687518034

## Show

In [153]:
path_obj.candidates

,ra,dec,ang_size,mag,separation,P_O,p_xO,P_Ox
2,263.65963,-50.76407,1.75,16.041294,27.790394,0.837299,0.000463,0.836771
4,263.65821,-50.76454,0.20,20.431294,31.431113,0.005918,0.000460,0.005876
7,263.65972,-50.76561,0.15,21.261294,30.916794,0.002639,0.000460,0.002620
9,263.66119,-50.76641,0.21,20.151294,30.577024,0.007842,0.000460,0.007786
13,263.66322,-50.76732,0.63,21.791294,30.541401,0.001609,0.000460,0.001597
18,263.66128,-50.76787,0.24,19.571294,34.597137,0.014254,0.000460,0.014152
29,263.66560,-50.77039,0.28,18.861294,38.722546,0.030439,0.000460,0.030221
